# Working with tabular data in Julia. ([DataFrames.jl](https://github.com/JuliaData/DataFrames.jl/tree/main)).

## What this notebook covers:

- **Creating** a DataFrame from scratch.
- **Reading** data from a CSV file.
- **Basic information** about the DataFrame.
- **Selecting specific columns** using `select`.
- **Filtering rows** based on conditions.
- **Subsetting rows** by index.
- **Adding a new column**.
- **Deleting** a column.
- **Renaming columns**.
- **Sorting** the DataFrame by a specific column.
- **Grouping by a column** and performing aggregation.
- **Looping** through groups and printing details.
- **Writing** the DataFrame to a CSV file.

#### 1. Install necessary packages

In [ ]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")

#### 2. Import the required packages

In [ ]:
using DataFrames
using CSV

#### 3. Create a DataFrame

In [ ]:
df = DataFrame(
    Resource = ["solar", "wind", "hydro", "nuclear", "coal", "natural gas", "biofuel"],
    Capacity = [500, 200, 10, 1000, 500, 5000, 50],
    Fuel_type = ["None", "None", "None", "Uranium", "Coal", "Natural Gas", "None"],
    Location = ["California", "California", "Oregon", "Georgia", "Virginia", "Texas", "California"]
)

#### 4. Load a CSV file into a DataFrame

In [ ]:
filepath = "uc_data/Demand.csv"
df = CSV.read(filepath, DataFrame)
# df = DataFrame(CSV.File(filepath))
# df = CSV.File(filepath) |> DataFrame

#### 5. Work with DataFrames

In [ ]:
# Load the CSV file into a DataFrame
df = CSV.read("uc_data/Generators_data.csv", DataFrame)
# Display the DataFrame
display(df)

To extract the columns of a `DataFrame` directly (i.e. without copying) you can use one of the following syntaxes: `df.T`, `df."T"`, `df[!, :T]` or `df[!, "T"]` (a copy is made when usign `df[:,T]`):

In [ ]:
df."Existing_Cap_MW"

You can obtain a vector of column names of the `DataFrames` as `Strings` using the `names` function:

In [ ]:
names(df)

If you were interested in element types of the columns, you can use the `eachcol` function to iterate over the columns, and `eltype` function to get the desired output:

In [ ]:
eltype.(eachcol(df))

Other useful functions:

In [ ]:
@show size(df)  # dataframe dimensions
@show size(df, 1)   # dataframe dimension along first axis
@show size(df, 2)   # dataframe dimension along second axis
@show nrow(df)  # number of rows
@show ncol(df)  # number of columns

Get basis statistcs of the `DataFrame`:

In [ ]:
describe(df)  

Visualize the first/last few rows of the `DataFrame`:

In [ ]:
first(df,6) # first 6 rows

In [ ]:
last(df,6)  # last 6 rows

Select subsets of a `DataFrame` (Ref: [Indexing](https://dataframes.juliadata.org/stable/lib/indexing/)):

In [ ]:
df[1:2, [1,3]]

In [ ]:
df[1:3, 1:4]

In [ ]:
df[1:3, [:Resource, :Existing_Cap_MW, :region]]

In [ ]:
df_selected = select(df, Not([:Fuel, :region, :num_units, :Existing_Cap_MW]))
display(df_selected)


Filter rows based on a condition:

In [ ]:
df_filtered = filter(row -> row.Existing_Cap_MW > 200, df)
display(df_filtered)

Change the data stored in a `DataFrame`:

In [ ]:
new_val = zeros(Int, nrow(df))
df1 = copy(df)
df1.Cap_size = new_val
df1

Add a new column to the `DataFrame`:

In [ ]:
df.Existing_Cap_GW = df.Existing_Cap_MW ./ 1000
display(df)


Delete a column:

In [ ]:
select!(df, Not(:zone))
display(df)

Rename a column:

In [ ]:
rename!(df, :region => :Location)
display(df)

Sort the `DataFrame` by the column `Existing_Cap_MW` in descending order:

In [ ]:
df_sorted = sort(df, :Existing_Cap_MW, rev=true)
display(df_sorted)

Group by `Location` and calculate mean capacity per location:

In [ ]:
Pkg.add("Statistics")
using Statistics

In [ ]:
df_grouped = combine(groupby(df, :Fuel), :Inv_cost_per_MWyr => mean => :Avg_Inv_cost_per_MWyr)
display(df_grouped)

Use a loop to print the details of each group:

In [ ]:
for fuel in unique(df.Fuel)
    println("Fuel: $fuel")
    fuel_rows = filter(row -> row.Fuel == fuel, df)
    display(fuel_rows)
end

Loop through `DataFrame` rows:

In [ ]:
cap_therm = Float64[]
for row in eachrow(df)
    if row.THERM == 1
        push!(cap_therm, row.Existing_Cap_MW)
    end
end
cap_therm

In [ ]:
cap_therm = [resource.Existing_Cap_MW for resource in eachrow(df) if resource.THERM == 1]

In [ ]:
df[df.THERM .== 1, :Existing_Cap_MW]

#### 6. Write the updated DataFrame to a CSV file

In [ ]:
CSV.write("generators_data_modified.csv", df, writeheader=true)

### Exercise 1
Read data stored in a gzip-compressed file `example8.csv.gz` into a `DataFrame` called `df`.

In [ ]:
# write your solution here

<details>
<summary>Solution</summary>

```julia
using CSV
using DataFrames

df = CSV.read("example8.csv.gz", DataFrame)
```

### Exercise 2
Get number of rows, columns, column names and summary statistics of the `df` data frame from exercise 10.

In [ ]:
# write your solution here

<details>
<summary>Solution</summary>

```julia
julia> nrow(df)
4
```
```julia
julia> ncol(df)
2
```
```julia
julia> names(df)
2-element Vector{String}:
 "number"
 "square"
 ```
```julia
julia> describe(df)
2×7 DataFrame
 Row │ variable  mean     min    median   max    nmissing  eltype
     │ Symbol    Float64  Int64  Float64  Int64  Int64     DataType
─────┼──────────────────────────────────────────────────────────────
   1 │ number       2.5       1      2.5      4         0  Int64
   2 │ square       7.75      2      6.5     16         0  Int64
```


### Exercise 3
Add a column to `df` data frame with label "name_string" containing string representation of numbers in column number, i.e. ["one", "two", "three", "four"].

In [ ]:
# write your solution here

<details>
<summary>Solution</summary>

```bash
julia> df."name string" = ["one", "two", "three", "four"]
4-element Vector{String}:
 "one"
 "two"
 "three"
 "four"
```

```bash
julia> df
4×3 DataFrame
 Row │ number  square  name string
     │ Int64   Int64   String
─────┼─────────────────────────────
   1 │      1       2  one
   2 │      2       4  two
   3 │      3       9  three
   4 │      4      16  four
```

### Exercise 4
Check if `df` contains column `square2`.

In [ ]:
# write your solution here

<details>
<summary>Solution</summary>


```bash
hasproperty(df, :square2)
```